# 拡張思考をいつ、どのように使用するか

このノートブックでは、Claude 3.7 Sonnet の拡張思考機能についてさらに詳しく調べ、次の点について検討します。

1. タスクの複雑さの分類フレームワーク
2. 拡張思考をいつ使用するかの決定木
3. 適切な使用例と不要な使用例の例
4. さまざまなタスク タイプでのパフォーマンス ベンチマーク
5. コストへの影響と最適化戦略

最後には、拡張思考が有益な場合と、特定のアプリケーションで拡張思考の使用を最適化する方法を判断するための体系的なアプローチを習得します。

> **注**: このレッスンでは、レッスン 1 で開発したユーティリティ関数を使用します。`claude_utils.py` モジュールには、Bedrock クライアントの作成、拡張思考の有無にかかわらず Claude の呼び出し、応答の表示を行うためのヘルパー関数が含まれています。これにより、定型コードを繰り返すのではなく、拡張思考をいつ、どのように使用するかというコア コンセプトに集中できます。
>
> レッスン 1 をまだ完了していない場合は、まずレッスン 1 を復習して、これらのユーティリティ関数の動作を理解してください。または、`claude_utils.py` ファイルを直接調べて、実装の詳細を確認することもできます。

In [ ]:
# Import required libraries
import boto3
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import display, Markdown, HTML
import claude_utils

# Configure plot styling
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)
pd.set_option('display.max_colwidth', None)

In [ ]:
# Set up the Bedrock clients using our utility module
REGION = 'us-west-2'  # Change to your preferred region
bedrock, bedrock_runtime = claude_utils.create_bedrock_clients(REGION)

# Claude 3.7 Sonnet model ID (consistent with Lesson 1)
CLAUDE_37_SONNET_MODEL_ID = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'

# Verify model availability
claude_utils.verify_model_availability(bedrock, CLAUDE_37_SONNET_MODEL_ID)

## 1. タスクの複雑さの分類フレームワーク

拡張思考が有益な場合を判断するには、まずタスクの複雑さを分類するフレームワークが必要です。これにより、拡張思考をいつ使用するか、推論にどのくらいの予算を割り当てるかについて体系的な決定を下すことができます。

このフレームワークでは、タスクを 4 つのレベルの複雑さに分類します。

1. **シンプル**: 単純な事実のクエリ、基本的な情報検索、簡単な計算
2. **中程度**: 複数ステップの推論、中程度の数学の問題、基本的な分析タスク
3. **複雑**: 詳細な分析、複雑な推論チェーン、制約の問題
4. **非常に複雑**: システム設計、高度な数学的証明、多段階の問題解決

タスクを複雑さに基づいて自動的に分類できる分類関数を実装しましょう。

In [ ]:
def classify_task_complexity(prompt, model_id='us.anthropic.claude-3-5-haiku-20241022-v1:0'):
    """
    Use Claude 3.5 Haiku to quickly classify the complexity of a task
    
    Args:
        prompt (str): The user prompt to classify
        model_id (str): The model ID to use for classification (defaults to Claude 3.5 Haiku for speed/cost)
        
    Returns:
        str: Complexity classification ('simple', 'medium', 'complex', or 'very_complex')
    """
    system_prompt = [
        {
            "text": """You are a task complexity classifier. Classify the complexity of the given task into one of these categories: 'simple', 'medium', 'complex', or 'very_complex'. 

Here are examples of each complexity level:
- simple: "What is the capital of France?", "Calculate 25% of 80", "Summarize this short paragraph in one sentence"
- medium: "A man has 53 socks in his drawer: 21 blue, 15 black and 17 red. How many socks must he take out to guarantee a black pair?", "Explain the greenhouse effect and its impact on climate"
- complex: "Design a ride-sharing service that optimizes for driver availability and route efficiency", "Analyze the causes and economic impacts of the 2008 financial crisis"
- very_complex: "Given a graph with n vertices and m edges, design an O(n+m) algorithm to find all bridges", "Design a quantum computing algorithm to solve the traveling salesman problem"

Respond with only the category name, nothing else."""
        }
    ]

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "text": f"Classify the complexity of this task: {prompt}"
                }
            ]
        }
    ]

    try:
        start_time = time.time()
        response = bedrock_runtime.converse(
            modelId=model_id,
            messages=messages,
            system=system_prompt,
            inferenceConfig={
                "temperature": 0,
                "maxTokens": 10  # We only need a short response
            }
        )
        elapsed_time = time.time() - start_time

        # Extract the classification
        result = None
        if response.get('output', {}).get('message', {}).get('content'):
            content_blocks = response['output']['message']['content']
            for block in content_blocks:
                if 'text' in block:
                    result = block['text'].strip().lower()
                    break

        # Ensure the result is one of our expected categories
        valid_categories = ['simple', 'medium', 'complex', 'very_complex']
        if result not in valid_categories:
            result = 'medium'  # Default to medium if unexpected response

        # Calculate approx cost
        tokens = response['usage']['totalTokens']
        cost = tokens * 0.00000125  # Assuming $0.00125 per 1K tokens for Haiku
        
        print(f"Classification: {result} (determined in {elapsed_time:.2f}s, {tokens} tokens, ${cost:.6f})")
        return result

    except Exception as e:
        print(f"Error classifying task complexity: {e}")
        return "medium"  # Default to medium complexity if there's an error

### タスク複雑度分類子の理解

`classify_task_complexity` 関数は、ユーザー プロンプトの複雑度を自動的に分類する効率的な方法です。仕組みは次のとおりです。

1. **より小さなモデルを活用**: この分類ステップでは、Claude 3.7 Sonnet ではなく **Claude 3.5 Haiku** を使用します。これは、この単純な決定タスクではより高速でコスト効率が高いためです。

2. **分類フレームワーク**: この関数は、4 つの複雑度カテゴリ (シンプル、中程度、複雑、非常に複雑) を定義する明示的な指示と、それぞれの例を添えて、プロンプトを Claude に送信します。

3. **効率性に関する考慮事項**: この関数は、次の方法で速度とコストが最適化されています:
- より小さなモデルを使用する **(Claude Haiku 3.5)**
- 確定的な応答のために温度を 0 に設定する
- 出力を 10 トークンに制限する
- カテゴリ名のみを要求する

4. **実際のアプリケーション**: これをワークフローの「トリアージ」ステップと考えてください。CPU スケジューラがさまざまなタスクに割り当てる処理時間を決定する方法に似ています。この初期評価により、適切な「思考リソース」を手元のタスクに割り当てることができます。

このアプローチにより、「思考パイプライン」が作成され、プロセスの各段階に適切なツールを使用して、タスクの要求に基づいて Claude の推論機能を効率的に割り当てることができます。

In [ ]:
# Define example tasks of varying complexity
example_tasks = {
    "simple_1": "What is the capital of France?",
    "simple_2": "What is 15% of 200?",

    "medium_1": "A man has 53 socks in his drawer: 21 identical blue, 15 identical black and 17 identical red. The lights are out, and he is completely in the dark. How many socks must he take out to make 100 percent certain he has at least one pair of black socks?",
    "medium_2": "Compare and contrast reinforcement learning and supervised learning in AI.",

    "complex_1": "Design a system for a ride-sharing service that optimizes for driver availability, passenger wait times, and route efficiency. Include considerations for peak hours, variable pricing, and geographic distribution of drivers.",
    "complex_2": "Analyze the causes and potential solutions to the prisoner's dilemma in game theory, including real-world applications and limitations.",

    "very_complex_1": "Given a graph G with n vertices and m edges, design an algorithm to find all bridges in the graph in O(n+m) time. A bridge is defined as an edge whose removal increases the number of connected components in the graph. Provide pseudocode and explain why your algorithm achieves the desired time complexity.",
    "very_complex_2": "Develop a comprehensive framework for a central bank to implement and manage a central bank digital currency (CBDC), addressing technological architecture, monetary policy implications, privacy concerns, and financial inclusion aspects."
}

# Test the classifier on our examples
print("Testing Claude Haiku 3.5 task complexity classifier...\n")
results = {}

for label, task in example_tasks.items():
    # Show abbreviated prompt if too long
    display_prompt = task if len(task) < 100 else task[:97] + "..."
    print(f"Task {label}: \"{display_prompt}\"")
    complexity = classify_task_complexity(task)
    results[label] = complexity
    print("-" * 80 + "\n")

# Display summary as a DataFrame
#summary_df = pd.DataFrame([(k, v) for k, v in results.items()], 
#                        columns=['Task', 'Classified Complexity'])
#display(summary_df)

## 2. 拡張思考を使用するタイミングの決定ツリー

タスクの複雑性フレームワークに基づいて、次の事項を決定するのに役立つ決定ツリーを作成できます。
1. 拡張思考を使用するかどうか
2. 推論予算をどれだけ割り当てるか

決定ツリーでは、次の点を考慮します。
- タスクの複雑性
- パフォーマンス要件
- 時間的制約
- コストの考慮事項

決定ツリーの視覚化を以下に示します。

![Decision Tree](./images/lesson2/complexity.png)

### ここで、拡張思考を使用するかどうか、およびどの予算を割り当てるかを自動的に決定する関数を作成しましょう。

In [ ]:
def determine_extended_thinking_strategy(prompt, time_sensitive=False):
    """
    Determine whether to use extended thinking and what budget to allocate
    based on task complexity and time sensitivity
    
    Args:
        prompt (str): The user prompt
        time_sensitive (bool): Whether the task is time-sensitive
        
    Returns:
        dict: Strategy with 'use_extended_thinking' and 'reasoning_budget' keys
    """
    # First, classify the task complexity
    complexity = classify_task_complexity(prompt)
    
    # Define reasoning budget ranges for each complexity level
    budget_ranges = {
        'simple': (0, 0),  # No extended thinking for simple tasks
        'medium': (1024, 2048),
        'complex': (2048, 8192),
        'very_complex': (8192, 16384)
    }
    
    # Determine whether to use extended thinking based on complexity and time sensitivity
    use_extended_thinking = True
    
    if complexity == 'simple':
        use_extended_thinking = False
    elif complexity == 'medium' and time_sensitive:
        use_extended_thinking = False
    
    # Determine reasoning budget (if using extended thinking)
    if use_extended_thinking:
        min_budget, max_budget = budget_ranges[complexity]
        
        # Use the lower end of the range if time_sensitive, otherwise use the middle
        if time_sensitive:
            reasoning_budget = min_budget
        else:
            reasoning_budget = (min_budget + max_budget) // 2
    else:
        reasoning_budget = 0
    
    strategy = {
        'complexity': complexity,
        'use_extended_thinking': use_extended_thinking,
        'reasoning_budget': reasoning_budget,
        'time_sensitive': time_sensitive
    }
    
    return strategy

### 拡張思考戦略関数の理解

`determine_extended_thinking_strategy` 関数は、意思決定ツリー ロジックを適用する自動意思決定システムとして機能します。まずタスクの複雑さを分類し、次に複雑さと時間的制約の両方に基づいて拡張思考を使用するかどうか、および割り当てる推論予算を決定します。スマート リソース マネージャーのように、タスクの要求に基づいて適切な量の「思考力」を使用して、タスクを適切な処理パイプラインに効率的にルーティングします。

## 3. 適切な使用例と拡張思考が不要な例

フレームワークと意思決定ツリーができたので、拡張思考が有益な場合と不要な場合の具体的な例を調べてみましょう。実際のプロンプトを使用して両方のシナリオをテストし、結果を比較します。

In [ ]:
def test_with_and_without_extended_thinking(prompt, max_tokens=500):
    """
    Test a prompt with and without extended thinking and compare the results
    
    Args:
        prompt (str): The prompt to test
        max_tokens (int): Maximum tokens for the response
        
    Returns:
        dict: Results including both responses and metrics
    """
    print(f"Testing prompt: {prompt[:100]}..." if len(prompt) > 100 else f"Testing prompt: {prompt}")
    
    # Determine optimal strategy
    strategy = determine_extended_thinking_strategy(prompt)
    print(f"Strategy: Complexity={strategy['complexity']}, Use Extended Thinking={strategy['use_extended_thinking']}, Budget={strategy['reasoning_budget']}")
    
    # Test without extended thinking
    print("\nTesting WITHOUT extended thinking...")
    standard_response = claude_utils.invoke_claude(
        bedrock_runtime,
        prompt, 
        CLAUDE_37_SONNET_MODEL_ID, 
        enable_reasoning=False,
        max_tokens=max_tokens
    )
    
    # Test with extended thinking (if recommended)
    reasoning_response = None
    if strategy['use_extended_thinking']:
        print("\nTesting WITH extended thinking...")
        reasoning_response = claude_utils.invoke_claude(
            bedrock_runtime,
            prompt, 
            CLAUDE_37_SONNET_MODEL_ID, 
            enable_reasoning=True,
            reasoning_budget=strategy['reasoning_budget'],
            max_tokens=max_tokens
        )
    
    # Display results
    standard_result = claude_utils.extract_response_content(standard_response)
    print("\n--- Standard Mode Result ---")
    standard_time = standard_response.get('_elapsed_time', 0)
    standard_tokens = standard_response.get('usage', {}).get('totalTokens', 0)
    standard_cost = (standard_response.get('usage', {}).get('inputTokens', 0) * 0.000003) + \
                   (standard_response.get('usage', {}).get('outputTokens', 0) * 0.000005)
    
    print(f"Time: {standard_time:.2f}s, Tokens: {standard_tokens}, Cost: ${standard_cost:.6f}")
    display(Markdown(f"**Standard Mode Result:**\n{standard_result[:500]}..."))
    
    # Only show extended thinking results if we used it
    reasoning_result = None
    if reasoning_response:
        reasoning_result = claude_utils.extract_response_content(reasoning_response)
        print("\n--- Extended Thinking Mode Result ---")
        reasoning_time = reasoning_response.get('_elapsed_time', 0)
        reasoning_tokens = reasoning_response.get('usage', {}).get('totalTokens', 0)
        reasoning_cost = (reasoning_response.get('usage', {}).get('inputTokens', 0) * 0.000003) + \
                        (reasoning_response.get('usage', {}).get('outputTokens', 0) * 0.000015)
        
        print(f"Time: {reasoning_time:.2f}s, Tokens: {reasoning_tokens}, Cost: ${reasoning_cost:.6f}")
        display(Markdown(f"**Extended Thinking Mode Result:**\n{reasoning_result[:500]}..."))
    
    return {
        'strategy': strategy,
        'standard_response': standard_response,
        'reasoning_response': reasoning_response,
        'standard_result': standard_result,
        'reasoning_result': reasoning_result
    }

### 次に、拡張思考の適切な使用例と不適切な使用例をいくつかテストしてみましょう。

In [ ]:
# Example 1: Simple factual query (should NOT use extended thinking)
simple_query = "What are the three primary colors of light?"
simple_results = test_with_and_without_extended_thinking(simple_query)
print("\n" + "="*80 + "\n")

# Example 2: Complex reasoning task (SHOULD use extended thinking)
complex_query = """
Analyze the knapsack problem in computer science. Explain the dynamic programming approach 
to solve it, provide pseudocode, and analyze the time and space complexity. 
Also explain when a greedy approach might work and when it would fail.
"""
complex_results = test_with_and_without_extended_thinking(complex_query, max_tokens=800)
print("\n" + "="*80 + "\n")

# Example 3: Medium complexity task (may use extended thinking if not time-sensitive)
medium_query = """
Compare and contrast supervised learning and unsupervised learning in machine learning. 
Give examples of algorithms in each category and scenarios where one would be preferred over the other.
"""
medium_results = test_with_and_without_extended_thinking(medium_query)

## 4. さまざまなタスク タイプでのパフォーマンス ベンチマーク

個々の例を確認したので、拡張思考の有無にかかわらず、さまざまなタスク タイプで Claude のパフォーマンスを体系的にベンチマークしてみましょう。いくつかの主要な指標を測定します:

1. **応答品質** (定性評価)
2. **応答生成時間** (レイテンシ)
3. **トークン使用量** (および関連コスト)
4. **効率** (1 秒あたりのトークン数)

このベンチマークにより、さまざまなタスク タイプで拡張思考と標準モードを使用することのトレードオフを定量化できます。

In [ ]:
def run_benchmarking_comparison(tasks, max_tokens=500):
    """
    Run a systematic benchmarking comparison across different task types
    
    Args:
        tasks (dict): Dictionary of task labels to prompts
        max_tokens (int): Maximum tokens for responses
        
    Returns:
        pd.DataFrame: Benchmarking results
    """
    results = []
    
    for label, prompt in tasks.items():
        print(f"\nBenchmarking task: {label}")
        print("-" * 50)
        
        # 1. Test without extended thinking
        print(f"Testing standard mode...")
        standard_start = time.time()
        standard_response = claude_utils.invoke_claude(
            bedrock_runtime,
            prompt, 
            CLAUDE_37_SONNET_MODEL_ID, 
            enable_reasoning=False,
            max_tokens=max_tokens
        )
        standard_time = time.time() - standard_start
        
        standard_input_tokens = standard_response.get('usage', {}).get('inputTokens', 0)
        standard_output_tokens = standard_response.get('usage', {}).get('outputTokens', 0)
        standard_total_tokens = standard_response.get('usage', {}).get('totalTokens', 0)
        standard_cost = (standard_input_tokens * 0.000003) + (standard_output_tokens * 0.000015)
        
        # 2. Test with extended thinking
        print(f"Testing extended thinking mode...")
        # Determine appropriate budget based on complexity
        complexity = classify_task_complexity(prompt)
        budget_map = {
            'simple': 1024,
            'medium': 2048,
            'complex': 4096,
            'very_complex': 8192
        }
        budget = budget_map.get(complexity, 2048)
        
        reasoning_start = time.time()
        reasoning_response = claude_utils.invoke_claude(
            bedrock_runtime,
            prompt, 
            CLAUDE_37_SONNET_MODEL_ID, 
            enable_reasoning=True,
            reasoning_budget=budget,
            max_tokens=max_tokens
        )
        reasoning_time = time.time() - reasoning_start
        
        reasoning_input_tokens = reasoning_response.get('usage', {}).get('inputTokens', 0)
        reasoning_output_tokens = reasoning_response.get('usage', {}).get('outputTokens', 0)
        reasoning_total_tokens = reasoning_response.get('usage', {}).get('totalTokens', 0)
        reasoning_cost = (reasoning_input_tokens * 0.000003) + (reasoning_output_tokens * 0.000015)
        
        # Calculate efficiency metrics
        standard_efficiency = standard_total_tokens / standard_time if standard_time > 0 else 0
        reasoning_efficiency = reasoning_total_tokens / reasoning_time if reasoning_time > 0 else 0
        
        # Collect results
        result = {
            'Task': label,
            'Complexity': complexity,
            'Standard_Time': standard_time,
            'Standard_Tokens': standard_total_tokens,
            'Standard_Cost': standard_cost,
            'Standard_Efficiency': standard_efficiency,
            'Reasoning_Time': reasoning_time,
            'Reasoning_Tokens': reasoning_total_tokens,
            'Reasoning_Cost': reasoning_cost,
            'Reasoning_Efficiency': reasoning_efficiency,
            'Time_Ratio': reasoning_time / standard_time if standard_time > 0 else 0,
            'Cost_Ratio': reasoning_cost / standard_cost if standard_cost > 0 else 0
        }
        
        print(f"Results: {complexity} task")
        print(f"Standard: {standard_time:.2f}s, {standard_total_tokens} tokens, ${standard_cost:.6f}")
        print(f"Reasoning: {reasoning_time:.2f}s, {reasoning_total_tokens} tokens, ${reasoning_cost:.6f}")
        
        results.append(result)
    
    # Convert to DataFrame
    df = pd.DataFrame(results)
    return df

# Define benchmark tasks
benchmark_tasks = {
    "FactualQuery": "What are the major planets in our solar system?",
    "SimpleCalc": "If I have 5 apples and give away 2, then buy 3 more, how many do I have?",
    "MediumAnalysis": "Compare the advantages and disadvantages of renewable energy vs. fossil fuels.",
    "SockDrawer": "A man has 53 socks in his drawer: 21 identical blue, 15 identical black and 17 identical red. The lights are out, and he is completely in the dark. How many socks must he take out to make 100 percent certain he has at least one pair of black socks?",
    "ComplexDesign": "Design a system for coordinating autonomous delivery drones in an urban environment, considering obstacles, weather, traffic patterns, and regulatory compliance.",
    "AdvancedCoding": "Explain how you would implement a distributed system for real-time processing of financial transactions that ensures ACID properties while maintaining high throughput."
}

# Run the benchmarking comparison
benchmark_results = run_benchmarking_comparison(benchmark_tasks)

# Display results
display(benchmark_results)

# Visualize the results
plt.figure(figsize=(14, 10))

# 1. Time comparison
plt.subplot(2, 2, 1)
benchmark_results.plot(kind='bar', x='Task', y=['Standard_Time', 'Reasoning_Time'], ax=plt.gca())
plt.title('Response Time Comparison')
plt.ylabel('Time (seconds)')
plt.xticks(rotation=45)

# 2. Cost comparison
plt.subplot(2, 2, 2)
benchmark_results.plot(kind='bar', x='Task', y=['Standard_Cost', 'Reasoning_Cost'], ax=plt.gca())
plt.title('Cost Comparison')
plt.ylabel('Cost ($)')
plt.xticks(rotation=45)

# 3. Efficiency comparison
plt.subplot(2, 2, 3)
benchmark_results.plot(kind='bar', x='Task', y=['Standard_Efficiency', 'Reasoning_Efficiency'], ax=plt.gca())
plt.title('Efficiency Comparison (Tokens/Second)')
plt.ylabel('Tokens per Second')
plt.xticks(rotation=45)

# 4. Ratio analysis by complexity
plt.subplot(2, 2, 4)
complexity_order = ['simple', 'medium', 'complex', 'very_complex']
benchmark_results['Complexity'] = pd.Categorical(benchmark_results['Complexity'], categories=complexity_order, ordered=True)
benchmark_results.sort_values('Complexity', inplace=True)

# Create separate scatter plots for each ratio instead of trying to plot both at once
benchmark_results.plot(kind='scatter', x='Complexity', y='Time_Ratio', color='blue', label='Time Ratio', ax=plt.gca())
benchmark_results.plot(kind='scatter', x='Complexity', y='Cost_Ratio', color='red', label='Cost Ratio', ax=plt.gca())

plt.title('Extended Thinking/Standard Ratios by Complexity')
plt.ylabel('Ratio (Extended/Standard)')
plt.xticks(rotation=45)
plt.legend()

plt.tight_layout()
plt.show()

### ベンチマーク機能の理解

`run_benchmarking_comparison` 関数は、実験室として機能し、拡張思考の有無にかかわらず、さまざまなタスク タイプで Claude のパフォーマンスを体系的にテストします。タスクごとに、応答時間、トークンの使用、コスト、効率の指標を測定し、パフォーマンスのトレードオフを正確に定量化し、拡張思考が最も価値を発揮する場所を判断できます。これは、事例証拠を超えて、拡張思考をいつ使用するかについてのデータに基づくガイドラインを作成するのに役立つ、制御された実験と考えてください。

## 5. コストへの影響と最適化戦略

ベンチマーク結果に基づいて、Claude の拡張思考機能を使用する際のコストとパフォーマンスのトレードオフを最適化するための戦略をいくつか開発できます。

### コスト構造

Claude 3.7 Sonnet の価格は、標準思考モードと拡張思考モードの両方で一貫しています。
- **入力トークン**: 100 万トークンあたり 3 ドル
- **出力トークン**: 100 万トークンあたり 15 ドル (思考トークンを含む)

つまり、拡張思考は主に推論に使用される追加の出力トークンを通じてコストを増加させます。

### 最適化戦略

コストと利益のトレードオフを最適化するためのいくつかの戦略を検討してみましょう:

| 戦略 | 説明 | 使用する場合 |
|---------|-------------|--------------|
| タスクの複雑さのフィルタリング | 複雑なタスクと非常に複雑なタスクにのみ拡張思考を使用します | 分類子を適用して、単純なタスクを標準モードにルーティングします |
| 動的な予算割り当て | タスクの複雑さに基づいて推論予算を割り当てます | 複雑さに基づいて、予算を 1024 トークン (最小) から 16384 トークン以上にスケーリングします |
| 2 段階アプローチ | 最初に標準モードを使用し、必要な場合にのみ拡張思考を呼び出します | 不確実なケースまたは標準モードの信頼性が低い場合 |
| 類似タスクのバッチ処理 | 分類コストを償却するために類似タスクをグループ化します | 多数の類似リクエストを処理するアプリケーション (例: カスタマー サービス) の場合 |
| 推論予算の上限 | ROI 分析に基づいて推論予算の上限を設定する | 予算が高額になると収益が減少することが観察される場合 |

### 実装例: コスト最適化された拡張思考

以下は、コストとパフォーマンスのバランスをとる最適化されたアプローチの実装です。この機能は次のようになります:

1. 複雑性分類器を使用して、拡張思考が必要かどうかを判断します
2. 複雑性に基づいて適切な推論予算を割り当てます
3. 予算上限を適用して、トークンの過剰使用を防止します
4. 詳細な監視メトリックを通じて透明性を提供します

In [ ]:
def cost_optimized_invoke(prompt, time_sensitive=False, max_tokens=1000):
    """
    Invoke Claude with cost-optimized extended thinking
    
    Args:
        prompt (str): The user prompt
        time_sensitive (bool): Whether the task is time-sensitive
        max_tokens (int): Maximum tokens for the final response
        
    Returns:
        dict: Response and performance metrics
    """
    # Step 1: Determine strategy
    strategy = determine_extended_thinking_strategy(prompt, time_sensitive=time_sensitive)
    
    # Step 2: Apply budget caps based on ROI analysis
    # (These caps would ideally be determined through extensive benchmarking)
    budget_caps = {
        'simple': 0,
        'medium': 2048,
        'complex': 4096,
        'very_complex': 8192
    }
    
    if strategy['use_extended_thinking']:
        strategy['reasoning_budget'] = min(strategy['reasoning_budget'], budget_caps[strategy['complexity']])
    
    # Step 3: Invoke Claude with the optimized settings
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        prompt,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=strategy['use_extended_thinking'],
        reasoning_budget=strategy['reasoning_budget'],
        max_tokens=max_tokens
    )
    
    # Step 4: Calculate performance metrics
    elapsed_time = response.get('_elapsed_time', 0)
    input_tokens = response.get('usage', {}).get('inputTokens', 0)
    output_tokens = response.get('usage', {}).get('outputTokens', 0)
    total_tokens = response.get('usage', {}).get('totalTokens', 0)
    
    input_cost = input_tokens * 0.000003
    output_cost = output_tokens * 0.000005
    total_cost = input_cost + output_cost
    
    metrics = {
        'strategy': strategy,
        'elapsed_time': elapsed_time,
        'input_tokens': input_tokens,
        'output_tokens': output_tokens,
        'total_tokens': total_tokens,
        'input_cost': input_cost,
        'output_cost': output_cost,
        'total_cost': total_cost
    }
    
    # Add the metrics to the response for transparency
    response['_metrics'] = metrics
    
    return response

# Demonstrate the cost-optimized approach
print("Testing cost-optimized approach...")
test_prompt = """
A spaceship needs to visit 5 planets (A, B, C, D, and E) starting from Earth and then returning to Earth. 
The distances between each pair of locations are as follows (in light-years):
Earth-A: 5, Earth-B: 7, Earth-C: 8, Earth-D: 10, Earth-E: 12
A-B: 4, A-C: 6, A-D: 9, A-E: 11
B-C: 5, B-D: 8, B-E: 10
C-D: 6, C-E: 9
D-E: 7

What's the shortest possible route that visits each planet exactly once before returning to Earth?
"""

cost_optimized_response = cost_optimized_invoke(test_prompt)
claude_utils.display_claude_response(cost_optimized_response)

# Display the optimization metrics
metrics = cost_optimized_response.get('_metrics', {})
print("\nOptimization Metrics:")
print(f"Task Complexity: {metrics.get('strategy', {}).get('complexity', 'unknown')}")
print(f"Extended Thinking Used: {metrics.get('strategy', {}).get('use_extended_thinking', False)}")
print(f"Reasoning Budget: {metrics.get('strategy', {}).get('reasoning_budget', 0)} tokens")
print(f"Time: {metrics.get('elapsed_time', 0):.2f} seconds")
print(f"Cost: ${metrics.get('total_cost', 0):.6f}")

## 結論: 拡張思考を効果的に使用するためのフレームワーク

Claude の拡張思考機能をいつ使用するか、またその使用を最適化する方法を決定するための体系的なフレームワークを開発しました:

1. 自動化された分類器を使用して **タスクの複雑さを分類**
2. **決定木を適用**して、拡張思考が必要かどうかを判断
3. 複雑さに基づいて **適切な推論予算を割り当てる**
4. **パフォーマンス メトリックを監視し**、アプローチを継続的に改善

このフレームワークに従うことで、次のことが可能になります:
- 複雑なタスクの応答品質を向上させる
- 単純なタスクの不要なコストを回避する
- パフォーマンスとコストの考慮事項のバランスをとる
- 特定の要件に基づいてアプローチを調整する

次のノートブックでは、ここで確立した基盤に基づいて、推論予算の割り当てを最適化する方法をさらに詳しく検討します。